In [1]:
from bs4 import BeautifulSoup
import requests
from PIL import Image
from twilio.rest import Client

page = requests.get("https://www.imdb.com/movies-in-theaters/")
soup = BeautifulSoup(page.content)
movie_list = soup.find('div', class_='list detail sub-list') #get list of upcoming movies

def getRating(title_link): #locate and retrieve IMDb audience rating, given movie website link
    moviepage = requests.get("https://www.imdb.com" + title_link)
    msoup = BeautifulSoup(moviepage.content)
    rating = msoup.find('div', class_='ratingValue').strong['title']
    return(rating)

class Movie: #contains all relevant information for an individual movie
    def __init__(self, title, overview, rating, posterURL):
        self.title = title
        self.overview = overview
        self.rating = rating
        self.posterURL = posterURL
        
    def __str__(self):
        return(self.title.ljust(28) + self.rating.ljust(35))
    
    def advertise(self):
        string = "Looking for a good movie to watch on the weekend? Try" + self.title + "!\nIMDb:" \
                    + self.rating + "\nDescription:" + self.overview
        return string
        
class MovieList: #stores list of Movie objects
    def __init__(self, date):
        self.list = []
        self.date = date
        
    def __str__(self):
        string = (" "+self.date).center(60, '-') + "\n"
        for movie in self.list:
            string += str(movie) + "\n"
        return string
        
    def addMovie(self, movie):
        self.list.append(movie)
        
    def getBestRated(self):
        bestMovie = None
        bestRating = 0
        for movie in self.list:
            rating = float(movie.rating[:3]) #ratings are stored as strings in the form "X.X based on XXX user ratings"
            if rating > bestRating:
                bestMovie = movie
                bestRating = rating
        return bestMovie
        
    def sortByRating(self): #sort movies from highest-rated to lowest-rated
        sortedList = sorted(self.list, key = lambda x: float(x.rating[:3]), reverse=True)
        self.list = sortedList
        return self
        
upcomingMovies = MovieList(movie_list.h3.string) #initialize movie list using the date of release

for movie in movie_list.find_all('div'):
    if 'list_item' in movie['class']:
        movie_info = movie.find('td', class_='overview-top') #retrieve all information from this bad boy
        movie_title = movie_info.find('a').string
        movie_overview = movie_info.find('div', class_='outline').string
        title_link = movie_info.find('a')['href']
        movie_rating = getRating(title_link)
        
        poster_info = movie.find('td', id='img_primary')
        poster_link = poster_info.find('img')['src'] #get image url for movie poster

        upcomingMovies.addMovie(Movie(movie_title, movie_overview, movie_rating, poster_link))
        
print(upcomingMovies.getBestRated().advertise())

Looking for a good movie to watch on the weekend? Try Glass (2019)!
IMDb:7.7 based on 3,180 user ratings
Description:
    Security guard David Dunn uses his supernatural abilities to track Kevin Wendell Crumb, a disturbed man who has twenty-four personalities.                    


In [ ]:
#Create free trial Twilio account to send SMS text message
client = Client("AC3533d6c757d019aa687d500e26398811", #Account SID
                "f2a4597dc5209ddb78b6911033e20afb")   #Auth Token

bestRatedMovie = upcomingMovies.getBestRated()
client.messages.create(to="+15623217824",    #Your phone number
                       from_="+16573151159", #Account phone number
                       body=bestRatedMovie.advertise(),     #Send blurb about highest rated movie coming out soon
                       media_url=bestRatedMovie.posterURL)  #Send movie poster image

https://github.com/brendanbrzycki/MovieGuide.git

In [ ]:
#Create free trial Twilio account to send SMS text message
client = Client("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX", #Account SID
                "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")   #Auth Token

bestRatedMovie = upcomingMovies.getBestRated()
client.messages.create(to="+XXXXXXXXXXX",    #Your phone number
                       from_="+XXXXXXXXXXX", #Account phone number
                       body=bestRatedMovie.advertise(),     #Send blurb about highest rated movie coming out soon
                       media_url=bestRatedMovie.posterURL)  #Send movie poster image